In [1]:
import numpy as np
import math

## 1

Система координат камеры получается из мировой поворотом на 45
градусов вокруг оси z и трансляцией на 10 вдоль оси z. Внутренние параметры
камеры: fx=fy=400, cx=960, cy=540. Написать программу, которая выводит на экран
матрицу проекции камеры и целочисленные координаты пикселя на изображении,
соответствующего трехмерной точке с координатами (10, -10, 100) в мировой
системе координат.

In [2]:
fxy = 400
cx = 960
cy = 540

K = np.array([
    [fxy, 0, cx],
    [0, fxy, cy],
    [0, 0, 1]
])

alpha = 45 * math.pi / 180
sin_a = math.sin(alpha)
cos_a = math.cos(alpha)

R_T = np.array([
    [cos_a, -sin_a, 0, 0],
    [sin_a, cos_a, 0, 0],
    [0, 0, 1, 10]
])

In [3]:
world_point = np.array([10, -10, 100, 1])

uv1 = np.matmul(K, np.matmul(R_T, world_point))
uv1 /= uv1[2]
print('Pixel coordinates', 'u:', round(uv1[0]), 'v:', round(uv1[1]), sep='\n')

Pixel coordinates
u:
1011
v:
540


## 2

Используя прямое линейное преобразование (direct linear
transformation) и SVD, написать программу, которая принимает на вход множество
пар двумерных точек, и выводит на экран соответствующую этим парам матрицу
гомографии.

In [4]:
def generate_equations_for_points(points1, points2):
    a = np.zeros(shape=(points1.shape[0] * 2, 9), dtype=np.float64)
    i = 0
    for p1, p2 in zip(points1, points2):
        u1, v1 = p1
        u2, v2 = p2
        a[i] = np.array([-u1, -v1, -1, 0, 0, 0, u1 * u2, v1 * u2, u2], dtype=np.float64)
        a[i + 1] = np.array([0, 0, 0, -u1, -v1, -1, u1 * v2, v1 * v2, v1], dtype=np.float64)
        i += 2
    return a


def svd_solve(a):
    u, d, vt = np.linalg.svd(a)

    threshold = 1e-15
    for eigenvalue, right_vector in zip(d, vt.T):
        if eigenvalue > -threshold and eigenvalue < threshold:
            return right_vector

In [5]:
old_points = np.array([
    [0, 0],
    [4, 4],
    [4, 0],
    [0, 4],
    [2, 2],
    [1, 3]
])

new_points = old_points / 2

a = generate_equations_for_points(old_points, new_points)
print('Equation coeffs:\n', a)

solution = svd_solve(a).reshape(3, 3)

print('Solution (H):\n', solution)

Equation coeffs:
 [[ 0.   0.  -1.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -1.   0.   0.   0. ]
 [-4.  -4.  -1.   0.   0.   0.   8.   8.   2. ]
 [ 0.   0.   0.  -4.  -4.  -1.   8.   8.   4. ]
 [-4.   0.  -1.   0.   0.   0.   8.   0.   2. ]
 [ 0.   0.   0.  -4.   0.  -1.   0.   0.   0. ]
 [ 0.  -4.  -1.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  -4.  -1.   0.   8.   4. ]
 [-2.  -2.  -1.   0.   0.   0.   2.   2.   1. ]
 [ 0.   0.   0.  -2.  -2.  -1.   2.   2.   2. ]
 [-1.  -3.  -1.   0.   0.   0.   0.5  1.5  0.5]
 [ 0.   0.   0.  -1.  -3.  -1.   1.5  4.5  3. ]]
Solution (H):
 [[ 0.3175196  -0.19776827 -0.07852322]
 [-0.14550545  0.56760999  0.09728233]
 [ 0.01500594 -0.23752201  0.66666667]]
